In [ ]:
#<----- Click this play button to setup MocapNET v4!

#This is the Total Capture version that handles body, hands, gaze, face!
#It also has been rewritten from scratch in Python for your convenience.
#If you deploy this on your PC, run : jupyter notebook mocapnet4.ipynb
#   and remove --collab from the setup.sh invocation 2 lines below..!
import os
if (os.path.isfile("MocapNET.py")):
    !git pull
    exit
!git clone -b mnet4 https://github.com/FORTH-ModelBasedTracker/MocapNET.git
!MocapNET/src/python/mnet4/setup.sh --collab
os.chdir("MocapNET/src/python/mnet4")
print("MocapNET setup is finished, you can run the next cell now..")


In [ ]:
#<----- Click to get a video from the internet and then track it ( e.g. shuffle.webm )

#You can also upload your own by clicking the Files icon and using the menu on the left
#You need to put files in the /content/MocapNET/src/python/mnet4/ directory

#Make sure we are at the correct directory
import os
if (not os.path.isfile("mediapipeHolisticWebcamMocapNET.py")):
    os.chdir("MocapNET/src/python/mnet4")

!wget http://ammar.gr/mocapnet/shuffle.webm -O shuffle.webm

#Analyze the file shuffle.web through MediaPipe 2D + MocapNET 3D Pose Estimation!
!(python3 -m mediapipeHolisticWebcamMocapNET --from shuffle.webm --ik 0.001 99 99 --smooth 60 10 --all --save --plot --headless 2> /dev/null)
print("MocapNET video input processing finished, you can run the next cells now to see the results..")

#Outputs are :
#livelastRun3DHiRes.mp4 | A video showing the regressed output overlayed on RGB
#livelastPlot3DHiRes.mp4| A video plot of the retrieved BVH degrees of freedom
#out.bvh                | the extracted BVH file 
#2d_out.csv             | Input 2D joints used by MocapNET as a CSV file
#3d_out.csv             | Output 3D joints produced by MocapNET as a CSV file
#bvh_out.csv            | Output BVH angles produced by MocapNET as a CSV file


In [ ]:
#<----- Click to run a sign-language dataset without mediapipe from a dumped Openpose/JSON/CSV source

#Make sure we are at the correct directory
import os
if (not os.path.isfile("csvNET.py")):
    os.chdir("MocapNET/src/python/mnet4")

#Download a single sign (con0014) from SIGNUM
!wget http://ammar.gr/datasets/signumtest.zip -O signumtest.zip
!unzip -o signumtest.zip

#Analyze the file con0014/2dJoints_v1.4.csv through MediaPipe 2D + MocapNET 3D Pose Estimation!
!(python3 -m csvNET --from con0014/2dJoints_v1.4.csv --ik 0.001 99 99 --upperbody --reye --mouth --hands --save --plot --headless 2> /dev/null)
print("MocapNET video input processing finished, you can run the next cells now to see the results..")

In [ ]:
#<----- Click to see the generated BVH angle plot of the last MocapNET run!
from IPython.display import Video
import os
if (not os.path.isfile("livelastPlot3DHiRes.mp4")):
   print("Please execute the previous cells and wait for them to complete before seeing this visualization!")
   exit

Video("livelastPlot3DHiRes.mp4",embed=True)

In [ ]:
#<----- Click to see the generated visualization of the last MocapNET run!
from IPython.display import Video
import os
if (not os.path.isfile("livelastRun3DHiRes.mp4")):
   print("Please execute the previous cells and wait for them to complete before seeing this visualization!")
   exit

Video("livelastRun3DHiRes.mp4",embed=True)

In [ ]:
#<----- Click to download the generated BVH/3D files of the last MocapNET run!
import os 
from google.colab import files
if (not os.path.isfile("out.bvh")) or (not os.path.isfile("2d_out.csv")) or (not os.path.isfile("bvh_out.csv")) or (not os.path.isfile("3d_out.csv")):
    print("Please execute the previous cells and wait for them to complete before downloading output!")
else:
    print("Download Output Files!")
    files.download("out.bvh")
    files.download("2d_out.csv")
    files.download("3d_out.csv")
    files.download("bvh_out.csv")
#To render the 3D face get Blender from https://www.blender.org/
#Download http://ammar.gr/mocapnet/mnet4/face.blend and open it using Blender
#Download http://ammar.gr/mocapnet/mnet4/headerWithHeadAndOneMotion.bvh and open it using Blender at a 0.01 scale
#Run The Loaded Python Script
#Click on the armature and on the menu right click the orange rectangle
#Select headerWithHeadAndOneMotion as Source BVH
#Select newgirl as Target Obj
#Select a directory as a target for generated dataset
#Select the path to the regressed bvh_out.csv to load pre-generated dataset
#Click Just Render CSV Dataset



In [ ]:
#<----- Click to attempt MocapNET live demo by streaming your camera to Google Colab
#EXPERIMENTAL, performance is impacted because of low image throughput!

import cv2
import os
from colabStream import init_camera,take_frame,show_frame

  
from mediapipeHolisticWebcamMocapNET import MediaPipeHolistic
from MocapNET import easyMocapNETConstructor
from folderStream import resize_with_padding

# Create instances of MediaPipeHolistic and MocapNET
mp_holistic = MediaPipeHolistic(doMediapipeVisualization=False)
mnet = easyMocapNETConstructor(
    engine              = "onnx",
    doProfiling         = False,
    multiThreaded       = False,
    doHCDPostProcessing = True,
    hcdLearningRate     = 0.001,
    hcdEpochs           = 99,
    hcdIterations       = 99,
    bvhScale            = 1.0,
    doBody              = True,
    doFace              = False,
    doREye              = True,
    doMouth             = True,
    doHands             = True,
    addNoise            = 0.0
)


# init JavaScript code
init_camera()

while True:
    try:
        image = take_frame()
 
        image = resize_with_padding(image,1280,720)

        # Perform image processing with MediaPipeHolistic
        mocapNETInput, annotated_image = mp_holistic.convertImageToMocapNETInput(image)

        # Perform 3D joint prediction with MocapNET
        mocapNET3DOutput = mnet.predict3DJoints(mocapNETInput)

        # Visualize the results on the image
        #image_with_results = Image.fromarray(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
 
        from MocapNETVisualization import visualizeMocapNETEnsemble
        image,plotImage = visualizeMocapNETEnsemble(mnet,annotated_image,plotBVHChannels=False)
    
        show_frame(annotated_image)  # it replace previous image
        
    except Exception as err:
        print('Exception:', err)

In [ ]:
#D/L and compile a different version of MocapNET and evaluate it against CMUBVH test dataset
#Running this will take a while, but it is usefult to compare different trained MocapNET Models!
import os

#Download Test Set!
if (not os.path.isfile("dataset/generated/bvh_body_all_test.csv")):
   print("Downloading test set!")
   os.system("wget http://ammar.gr/datasets/testBody.zip && unzip testBody.zip")

#Download a different MocapNET build
!python3 -m getModelFromDatabase --get 319 #<- change number to try a different version

#Do Evaluation
!python3 -m evaluateMocapNET --config dataset/body_configuration.json --all body --skip 5 --engine onnx > lastEvaluationLog.txt
!tail -n 30 lastEvaluationLog.txt 

In [2]:
%%HTML
<!-- Activate this block to see a youtube video about rendering MocapNET results with Blender -->
<iframe width="560" height="315" src="https://youtube.com/embed/ooLRUS5j4AI"></iframe>

In [ ]:
"""

This library is provided under the FORTH license
https://github.com/FORTH-ModelBasedTracker/MocapNET/blob/master/license.txt

If you use this version of MocapNET for your research please consider citing : 

@inproceedings{Qammaz2023b,
  author = {Qammaz, Ammar and Argyros, Antonis},
  title = {A Unified Approach for Occlusion Tolerant 3D Facial Pose Capture and Gaze Estimation using MocapNETs},
  booktitle = {International Conference on Computer Vision Workshops (AMFG 2023 - ICCVW 2023), (to appear)},
  publisher = {IEEE},
  year = {2023},
  month = {October},
  address = {Paris, France},
  projects =  {VMWARE,I.C.HUMANS},
  pdflink = {http://users.ics.forth.gr/ argyros/mypapers/2023_10_AMFG_Qammaz.pdf}
}

@inproceedings{Qammaz2021,
  author = {Qammaz, Ammar and Argyros, Antonis A},
  title = {Towards Holistic Real-time Human 3D Pose Estimation using MocapNETs},
  booktitle = {British Machine Vision Conference (BMVC 2021)},
  publisher = {BMVA},
  year = {2021},
  month = {November},
  projects =  {I.C.HUMANS},
  videolink = {https://www.youtube.com/watch?v=aaLOSY_p6Zc}
}
""" 